# Test Suite for Refactored `prepare_transcripts.py`

This notebook tests the refactored preprocessing module with CHILDES sample data and verifies compatibility with alignment analysis scripts.

## Prerequisites

Before running this notebook, ensure you've:
1. Installed the package in editable mode: `pip install -e .`
2. Installed spaCy: `pip install spacy` [NOTE: is this necessary?]
3. Downloaded spaCy model: `python -m spacy download en_core_web_sm` [NOTE: is this necessary?]

## Data Location

Test files: `/Users/ndd697/Desktop/Github-Projects/llm-linguistic-alignment/src/align_test/data/CHILDES/`

---
## Setup: Import Libraries and Configure Paths

In [ ]:
import os 
import pandas as pd
import ast

# Import the refactored preprocessing module
from align_test.prepare_transcripts import prepare_transcripts

print("✓ Imports successful")


In [ ]:
# ============================================================
# CONFIGURATION: Set your data directories
# ============================================================

# Input: CHILDES data directory
CHILDES_DATA_DIR = '/Users/ndd697/Desktop/Github-Projects/llm-linguistic-alignment/src/align_test/data/CHILDES/'

# Output: Test output directories
OUTPUT_DIR_BASIC = './test_output_basic'
OUTPUT_DIR_SPACY = './test_output_spacy'
OUTPUT_DIR_STANFORD = './test_output_stanford'

# Create output directories
for dir_path in [OUTPUT_DIR_BASIC, OUTPUT_DIR_SPACY, OUTPUT_DIR_STANFORD]:
    os.makedirs(dir_path, exist_ok=True)
    print(f"✓ Created directory: {dir_path}")

---
## Verify Input Data

Check that the CHILDES directory exists and contains our test files.

In [ ]:
# Check if CHILDES directory exists
print(f"CHILDES Directory: {CHILDES_DATA_DIR}")
print(f"Exists: {os.path.exists(CHILDES_DATA_DIR)}")

if not os.path.exists(CHILDES_DATA_DIR):
    print("\n✗ Directory not found! Please update CHILDES_DATA_DIR above.")
else:
    # List files in directory
    files = [f for f in os.listdir(CHILDES_DATA_DIR) if f.endswith('.txt')]
    print(f"\n✓ Found {len(files)} .txt files:")
    for f in files:
        file_path = os.path.join(CHILDES_DATA_DIR, f)
        size_kb = os.path.getsize(file_path) / 1024
        print(f"  - {f} ({size_kb:.1f} KB)")

---
## Inspect Raw Input Files

Let's look at the structure of the raw input files before preprocessing.

In [ ]:
# Load and display a sample input file
sample_file = os.path.join(CHILDES_DATA_DIR, 'time200-cond1.txt')

print(f"Reading: {os.path.basename(sample_file)}\n")

raw_df = pd.read_csv(sample_file, sep='\t', encoding='utf-8')

print(f"Columns: {raw_df.columns.tolist()}")
print(f"Rows: {len(raw_df)}")
print(f"\nFirst 5 rows:")
raw_df.head()

In [ ]:
# Show some sample content
print("Sample utterances:\n")
for i in range(min(5, len(raw_df))):
    print(f"{raw_df['participant'].iloc[i]}: {raw_df['content'].iloc[i]}")

---
## TEST 1: Basic Preprocessing (NLTK Only)

Test preprocessing with default NLTK POS tagger (fastest option).

In [ ]:
print("="*60)
print("TEST 1: Basic Preprocessing (NLTK only)")
print("="*60)

# Run preprocessing with minimal options
results_basic = prepare_transcripts(
    input_files=CHILDES_DATA_DIR,
    output_file_directory=OUTPUT_DIR_BASIC,
    run_spell_check=False,  # Disable for faster testing
    minwords=2,
    add_additional_tags=False,  # NLTK only
    input_as_directory=True
)

print(f"\n✓ Preprocessing complete!")
print(f"Total utterances processed: {len(results_basic)}")

In [ ]:
# Examine the output
print("Output columns:")
for col in results_basic.columns:
    print(f"  - {col}")

print(f"\nDataFrame shape: {results_basic.shape}")
results_basic.head(3)

In [ ]:
# Examine a single row in detail
print("Sample processed utterance:\n")
sample_row = results_basic.iloc[0]

print(f"Participant: {sample_row['participant']}")
print(f"Content: {sample_row['content']}")
print(f"\nToken (string): {sample_row['token'][:100]}...")
print(f"Type: {type(sample_row['token'])}")

# Parse and display
tokens = ast.literal_eval(sample_row['token'])
print(f"\nToken (parsed): {tokens}")
print(f"Type after parsing: {type(tokens)}")

### Validate Output Format

Check that the output format is compatible with alignment analysis scripts.

In [ ]:
# Load one of the saved output files
output_files = [f for f in os.listdir(OUTPUT_DIR_BASIC) 
                if f.endswith('.txt') and 'concatenated' not in f]

print(f"Output files created: {output_files}")

# Load the first file
test_file_path = os.path.join(OUTPUT_DIR_BASIC, output_files[0])
print(f"\nLoading: {output_files[0]}")

test_df = pd.read_csv(test_file_path, sep='\t', encoding='utf-8')
print(f"Rows loaded: {len(test_df)}")
print(f"Columns: {test_df.columns.tolist()}")

In [ ]:
# Test 1: Verify all columns are present
required_cols = ['participant', 'content', 'token', 'lemma', 'tagged_token', 'tagged_lemma', 'file']

print("Test 1: Required Columns")
print("-" * 40)
for col in required_cols:
    present = col in test_df.columns
    status = "✓" if present else "✗"
    print(f"{status} {col}")

all_present = all(col in test_df.columns for col in required_cols)
print(f"\nResult: {'✓ PASSED' if all_present else '✗ FAILED'}")

In [ ]:
# Test 2: Verify data types (should be strings)
list_columns = ['token', 'lemma', 'tagged_token', 'tagged_lemma']

print("Test 2: Data Types (should be strings)")
print("-" * 40)

all_strings = True
for col in list_columns:
    if col in test_df.columns:
        first_val = test_df[col].iloc[0]
        is_string = isinstance(first_val, str)
        status = "✓" if is_string else "✗"
        print(f"{status} {col}: {type(first_val).__name__}")
        if not is_string:
            all_strings = False

print(f"\nResult: {'✓ PASSED' if all_strings else '✗ FAILED'}")

In [ ]:
# Test 3: Verify ast.literal_eval compatibility
print("Test 3: ast.literal_eval Compatibility")
print("-" * 40)

all_parseable = True
for col in list_columns:
    if col in test_df.columns:
        try:
            parsed = ast.literal_eval(test_df[col].iloc[0])
            print(f"✓ {col}: Parses to {type(parsed).__name__}")
            print(f"  Sample: {str(parsed)[:60]}...")
        except Exception as e:
            print(f"✗ {col}: Parse failed - {e}")
            all_parseable = False

print(f"\nResult: {'✓ PASSED' if all_parseable else '✗ FAILED'}")

In [ ]:
# Test 4: Verify POS tag tuple format
print("Test 4: POS Tag Format")
print("-" * 40)

correct_format = True
for col in ['tagged_token', 'tagged_lemma']:
    if col in test_df.columns:
        try:
            parsed = ast.literal_eval(test_df[col].iloc[0])
            if parsed:
                is_tuple = isinstance(parsed[0], tuple)
                correct_length = len(parsed[0]) == 2 if is_tuple else False
                
                if is_tuple and correct_length:
                    print(f"✓ {col}: Correct format")
                    print(f"  Sample: {parsed[0]} (word, POS)")
                else:
                    print(f"✗ {col}: Incorrect format")
                    correct_format = False
        except Exception as e:
            print(f"✗ {col}: Format check failed - {e}")
            correct_format = False

print(f"\nResult: {'✓ PASSED' if correct_format else '✗ FAILED'}")

### TEST 1 Summary

In [ ]:
print("="*60)
print("TEST 1 SUMMARY: Basic Preprocessing")
print("="*60)

test1_passed = all_present and all_strings and all_parseable and correct_format

if test1_passed:
    print("\n✓ TEST 1 PASSED: Basic preprocessing works correctly!")
    print("\nOutput format is compatible with alignment analysis.")
else:
    print("\n✗ TEST 1 FAILED: Some checks did not pass.")
    print("Please review the test results above.")

---
## TEST 2: Preprocessing with spaCy

Test preprocessing with spaCy POS tagger (100x faster than Stanford).

In [ ]:
# Check if spaCy is available
try:
    import spacy
    print("✓ spaCy is installed")
    print("Note: Model will be auto-downloaded by prepare_transcripts() if needed")
    spacy_available = True
except ImportError:
    print("✗ spaCy not installed")
    print("Install with: pip install spacy")
    print("Will skip spaCy tests")
    spacy_available = False

In [ ]:
# Only run if spaCy is available
if spacy_available:
    print("="*60)
    print("TEST 2: Preprocessing with spaCy")
    print("="*60)
    
    # Run preprocessing with spaCy
    results_spacy = prepare_transcripts(
        input_files=CHILDES_DATA_DIR,
        output_file_directory=OUTPUT_DIR_SPACY,
        run_spell_check=False,
        minwords=2,
        add_additional_tags=True,
        tagger_type='spacy',  # Use spaCy
        input_as_directory=True
    )
    
    print(f"\n✓ Preprocessing with spaCy complete!")
    print(f"Total utterances processed: {len(results_spacy)}")
else:
    print("\nSkipping spaCy test (spaCy not available)")
    results_spacy = None

### TEST 2 Summary

In [ ]:
# Examine spaCy output (if available)
if results_spacy is not None:
    print("Output columns:")
    for col in results_spacy.columns:
        print(f"  - {col}")
    
    # Check for spaCy-specific columns
    has_spacy_cols = 'tagged_spacy_token' in results_spacy.columns and 'tagged_spacy_lemma' in results_spacy.columns
    
    if has_spacy_cols:
        print("\n✓ spaCy tagging columns present (tagged_spacy_token, tagged_spacy_lemma)")
        
        # Show sample spaCy tags
        sample_spacy_tag = ast.literal_eval(results_spacy['tagged_spacy_token'].iloc[0])
        print(f"\nSample spaCy tags:")
        for i, (word, tag) in enumerate(sample_spacy_tag[:5]):
            print(f"  {i+1}. ('{word}', '{tag}')")
    else:
        print("\n✗ spaCy tagging columns missing!")
    
    results_spacy.head(3)

In [ ]:
print("="*60)
print("TEST 2 SUMMARY: spaCy Preprocessing")
print("="*60)

if results_spacy is not None:
    test2_passed = has_spacy_cols and len(results_spacy) > 0
    
    if test2_passed:
        print("\n✓ TEST 2 PASSED: spaCy preprocessing works correctly!")
        print("\nspaCy tags are being generated and stored properly.")
    else:
        print("\n✗ TEST 2 FAILED: spaCy preprocessing had issues.")
else:
    print("\n⊘ TEST 2 SKIPPED: spaCy not available")
    test2_passed = None

### Compare NLTK tags vs spaCy tags

In [ ]:
# Compare NLTK tags vs spaCy tags for same utterance
if results_spacy is not None:
    print("Comparison: NLTK vs spaCy POS Tags")
    print("="*60)
    
    sample_row = results_spacy.iloc[0]
    
    nltk_tags = ast.literal_eval(sample_row['tagged_token'])
    spacy_tags = ast.literal_eval(sample_row['tagged_spacy_token'])
    
    print(f"Utterance: {sample_row['content']}\n")
    print(f"{'Word':<15} {'NLTK Tag':<10} {'spaCy Tag':<10} {'Same?':<10}")
    print("-" * 50)
    
    for (word_n, tag_n), (word_s, tag_s) in zip(nltk_tags, spacy_tags):
        same = "✓" if tag_n == tag_s else "✗"
        print(f"{word_n:<15} {tag_n:<10} {tag_s:<10} {same:<10}")
    
    # Calculate agreement
    agreements = sum(1 for (_, t1), (_, t2) in zip(nltk_tags, spacy_tags) if t1 == t2)
    total = len(nltk_tags)
    agreement_pct = (agreements / total * 100) if total > 0 else 0
    
    print(f"\nAgreement: {agreements}/{total} ({agreement_pct:.1f}%)")



In [ ]:
# Compare NLTK tags vs spaCy tags across ALL utterances
if results_spacy is not None:
    print("="*60)
    print("COMPREHENSIVE COMPARISON: NLTK vs spaCy POS Tags")
    print("="*60)
    
    total_agreements = 0
    total_tokens = 0
    per_utterance_agreements = []
    
    # Calculate agreement across all utterances
    for idx in range(len(results_spacy)):
        sample_row = results_spacy.iloc[idx]
        
        nltk_tags = ast.literal_eval(sample_row['tagged_token'])
        spacy_tags = ast.literal_eval(sample_row['tagged_spacy_token'])
        
        if nltk_tags and spacy_tags and len(nltk_tags) == len(spacy_tags):
            agreements = sum(1 for (_, t1), (_, t2) in zip(nltk_tags, spacy_tags) if t1 == t2)
            total_agreements += agreements
            total_tokens += len(nltk_tags)
            
            # Track per-utterance agreement
            utterance_pct = (agreements / len(nltk_tags)) * 100
            per_utterance_agreements.append(utterance_pct)
    
    # Overall statistics
    overall_agreement_pct = (total_agreements / total_tokens * 100) if total_tokens > 0 else 0
    
    print(f"\n📊 OVERALL STATISTICS:")
    print(f"   Total tokens compared: {total_tokens}")
    print(f"   Agreements: {total_agreements}")
    print(f"   Disagreements: {total_tokens - total_agreements}")
    print(f"   Overall Agreement: {overall_agreement_pct:.1f}%")
    
    if per_utterance_agreements:
        import numpy as np
        print(f"\n   Per-utterance agreement:")
        print(f"      Mean: {np.mean(per_utterance_agreements):.1f}%")
        print(f"      Median: {np.median(per_utterance_agreements):.1f}%")
        print(f"      Min: {np.min(per_utterance_agreements):.1f}%")
        print(f"      Max: {np.max(per_utterance_agreements):.1f}%")
    
    # Show detailed examples
    print("\n" + "="*60)
    print("DETAILED EXAMPLES (First 3 utterances with >5 words)")
    print("="*60)
    
    examples_shown = 0
    for idx in range(len(results_spacy)):
        if examples_shown >= 3:
            break
            
        sample_row = results_spacy.iloc[idx]
        nltk_tags = ast.literal_eval(sample_row['tagged_token'])
        spacy_tags = ast.literal_eval(sample_row['tagged_spacy_token'])
        
        if len(nltk_tags) > 5 and len(spacy_tags) > 5:
            examples_shown += 1
            
            print(f"\n--- Example {examples_shown} ---")
            print(f"Source: {sample_row.get('file', 'unknown')}")
            print(f"Participant: {sample_row.get('participant', 'unknown')}")
            print(f"Utterance: {sample_row['content']}\n")
            print(f"{'Word':<15} {'NLTK':<10} {'spaCy':<10} {'Match':<8}")
            print("-" * 48)
            
            agreements = 0
            disagreements = []
            
            for (word_n, tag_n), (word_s, tag_s) in zip(nltk_tags, spacy_tags):
                match = "✓" if tag_n == tag_s else "✗"
                print(f"{word_n:<15} {tag_n:<10} {tag_s:<10} {match:<8}")
                
                if tag_n == tag_s:
                    agreements += 1
                else:
                    disagreements.append((word_n, tag_n, tag_s))
            
            total = len(nltk_tags)
            print(f"\nAgreement: {agreements}/{total} ({100*agreements/total:.1f}%)")
            
            if disagreements:
                print(f"Disagreements: {len(disagreements)}")
                for word, nltk_tag, spacy_tag in disagreements[:3]:  # Show first 3
                    print(f"  • '{word}': NLTK={nltk_tag}, spaCy={spacy_tag}")
    
    # Identify most common disagreements
    print("\n" + "="*60)
    print("MOST COMMON TAG DISAGREEMENTS")
    print("="*60)
    
    disagreement_counts = {}
    for idx in range(len(results_spacy)):
        sample_row = results_spacy.iloc[idx]
        nltk_tags = ast.literal_eval(sample_row['tagged_token'])
        spacy_tags = ast.literal_eval(sample_row['tagged_spacy_token'])
        
        if nltk_tags and spacy_tags:
            for (word, t1), (_, t2) in zip(nltk_tags, spacy_tags):
                if t1 != t2:
                    key = f"NLTK:{t1} vs spaCy:{t2}"
                    if key not in disagreement_counts:
                        disagreement_counts[key] = []
                    disagreement_counts[key].append(word)
    
    # Show top 10 disagreements
    if disagreement_counts:
        sorted_disagreements = sorted(disagreement_counts.items(), 
                                     key=lambda x: len(x[1]), 
                                     reverse=True)
        
        print("\nTop 10 tag disagreement patterns:")
        for i, (pattern, words) in enumerate(sorted_disagreements[:10], 1):
            example_words = ', '.join(list(set(words))[:3])  # Show up to 3 unique examples
            print(f"{i:2}. {pattern:<30} (n={len(words):3}) Examples: {example_words}")
    else:
        print("\n✓ Perfect agreement! No disagreements found.")
    
    print("\n" + "="*60)

---
## TEST 3: Preprocessing with Stanford

In [ ]:
import subprocess

# Step 1: Check if Java is installed
print("\n1. Checking Java installation...")
print("-" * 40)

java_available = False
try:
    result = subprocess.run(['java', '-version'], 
                          capture_output=True, 
                          text=True, 
                          timeout=5)
    
    # Check both stderr and stdout for Java version info
    output = result.stderr + result.stdout
    
    # Java typically outputs to stderr, and should contain "version"
    # Check return code AND output content
    if result.returncode == 0 and ('version' in output.lower() or 'openjdk' in output.lower()):
        # Extract version line (usually first line)
        version_lines = [line for line in output.split('\n') if line.strip()]
        if version_lines:
            java_version = version_lines[0]
            # Double-check it's not an error message
            if 'unable to locate' not in java_version.lower() and 'not found' not in java_version.lower():
                print(f"✓ Java is installed: {java_version}")
                java_available = True
            else:
                print("✗ Java not found")
                print(f"  Error: {java_version}")
    else:
        print("✗ Java not found or not working properly")
        if output.strip():
            print(f"  Output: {output.strip()[:100]}")
        
except FileNotFoundError:
    print("✗ Java command not found")
    print("  Java is not installed or not in PATH")
except subprocess.TimeoutExpired:
    print("✗ Java check timed out")
except Exception as e:
    print(f"✗ Error checking Java: {e}")

if not java_available:
    print("\n  Stanford POS Tagger requires Java to run")
    print("  Install Java from:")
    print("    - macOS: https://www.java.com/en/download/")
    print("    - macOS (alternative): brew install openjdk")
    print("    - Linux: sudo apt-get install default-jdk")
    print("    - Windows: https://www.java.com/en/download/")

In [ ]:
# Step 2: Check for Stanford POS Tagger files
print("\n2. Checking Stanford POS Tagger files...")
print("-" * 40)

stanford_available = False
stanford_pos_path = None
stanford_language_path = None

if java_available:
    # Common locations where users might have Stanford tagger
    common_locations = [
        os.path.expanduser("~/stanford-postagger"),
        os.path.expanduser("~/Downloads/stanford-postagger-full-2020-11-17"),
        "/usr/local/stanford-postagger",
        "./stanford-postagger",
    ]
    
    # Check if any common location exists
    for location in common_locations:
        if os.path.exists(location):
            # Check for required files
            jar_path = os.path.join(location, "stanford-postagger.jar")
            model_path = os.path.join(location, "models/english-left3words-distsim.tagger")
            
            if os.path.exists(jar_path) and os.path.exists(model_path):
                stanford_pos_path = location
                stanford_language_path = "models/english-left3words-distsim.tagger"
                stanford_available = True
                print(f"✓ Found Stanford tagger at: {location}")
                print(f"  JAR: {jar_path}")
                print(f"  Model: {model_path}")
                break
    
    if not stanford_available:
        print("✗ Stanford POS Tagger not found in common locations")
        print("\nTo use Stanford tagger:")
        print("  1. Download from: https://nlp.stanford.edu/software/tagger.shtml#Download")
        print("  2. Extract to a known location")
        print("  3. Update the paths below")
        print("\nCommon locations checked:")
        for loc in common_locations:
            print(f"  - {loc}")
else:
    print("⊘ Skipping (Java not available)")

In [ ]:
# Step 3: Manual path configuration (if not auto-detected)
print("\n3. Path Configuration")
print("-" * 40)

if java_available and not stanford_available:
    print("\n⚠️  Stanford tagger not auto-detected.")
    print("If you have Stanford tagger installed, specify paths below:")
    print("\nExample paths:")
    print("  stanford_pos_path = '/Users/yourname/stanford-postagger-full-2020-11-17'")
    print("  stanford_language_path = 'models/english-left3words-distsim.tagger'")
    
    # Uncomment and update these lines if you have Stanford tagger installed:
    # stanford_pos_path = "/path/to/your/stanford-postagger"
    # stanford_language_path = "models/english-left3words-distsim.tagger"
    # stanford_available = True
    
    if stanford_pos_path and stanford_language_path:
        # Validate the paths
        jar_path = os.path.join(stanford_pos_path, "stanford-postagger.jar")
        model_path = os.path.join(stanford_pos_path, stanford_language_path)
        
        if os.path.exists(jar_path) and os.path.exists(model_path):
            stanford_available = True
            print(f"✓ Manual configuration successful")
        else:
            print(f"✗ Invalid paths:")
            if not os.path.exists(jar_path):
                print(f"  JAR not found: {jar_path}")
            if not os.path.exists(model_path):
                print(f"  Model not found: {model_path}")
            stanford_available = False
elif stanford_available:
    print(f"✓ Using auto-detected paths:")
    print(f"  Base: {stanford_pos_path}")
    print(f"  Model: {stanford_language_path}")

In [ ]:
## MANUALLY ADD PATHS HERE IF NEEDED ##
# Example manual configuration (uncomment and set your paths)
stanford_pos_path = '/Users/ndd697/Desktop/Github-Projects/llm-linguistic-alignment/sandbox-prepare/stanford-postagger-full-2020-11-17'
stanford_language_path = '/Users/ndd697/Desktop/Github-Projects/llm-linguistic-alignment/sandbox-prepare/stanford-postagger-full-2020-11-17/models/english-left3words-distsim.tagger'

In [ ]:
# Step 4: Run Stanford tagging (if available)
print("\n4. Test Results")
print("-" * 40)

# RE-VALIDATE: Check if paths were manually set after Step 3
if not stanford_available and 'stanford_pos_path' in locals() and 'stanford_language_path' in locals():
    if stanford_pos_path is not None and stanford_language_path is not None:
        print("\n🔄 Detected manually configured paths. Validating...")
        
        # Normalize paths
        stanford_pos_path = os.path.normpath(os.path.expanduser(stanford_pos_path))
        stanford_language_path = os.path.normpath(stanford_language_path)
        
        # Check if stanford_language_path is absolute or relative
        if os.path.isabs(stanford_language_path):
            # It's an absolute path, use it directly
            model_path = stanford_language_path
        else:
            # It's relative to stanford_pos_path
            model_path = os.path.join(stanford_pos_path, stanford_language_path)
        
        jar_path = os.path.join(stanford_pos_path, "stanford-postagger.jar")
        
        print(f"  Checking JAR: {jar_path}")
        print(f"  Checking Model: {model_path}")
        
        if os.path.exists(jar_path) and os.path.exists(model_path):
            stanford_available = True
            print(f"  ✓ Manual configuration validated!")
            print(f"  ✓ Found JAR: {os.path.basename(jar_path)}")
            print(f"  ✓ Found Model: {os.path.basename(model_path)}")
            
            # Update stanford_language_path to be relative if it was given as absolute
            if os.path.isabs(stanford_language_path):
                # Convert to relative path from stanford_pos_path
                try:
                    stanford_language_path = os.path.relpath(model_path, stanford_pos_path)
                    print(f"  ℹ️  Converted to relative path: {stanford_language_path}")
                except ValueError:
                    # Can't make relative (e.g., different drives on Windows)
                    print(f"  ℹ️  Using absolute model path")
        else:
            print(f"  ✗ Validation failed:")
            if not os.path.exists(jar_path):
                print(f"    - JAR not found: {jar_path}")
            if not os.path.exists(model_path):
                print(f"    - Model not found: {model_path}")
            print(f"\n  💡 Tips:")
            print(f"    - stanford_pos_path should point to the Stanford tagger directory")
            print(f"    - stanford_language_path can be either:")
            print(f"      • Relative: 'models/english-left3words-distsim.tagger'")
            print(f"      • Absolute: '/full/path/to/models/english-left3words-distsim.tagger'")

# Now proceed with the test
if not java_available:
    print("\n⊘ TEST 3 SKIPPED: Java not installed")
    print("Stanford tagger requires Java to run.")
    stanford_test_passed = None
    
elif not stanford_available:
    print("\n⊘ TEST 3 SKIPPED: Stanford tagger not configured")
    print("Stanford tagger files not found or paths not specified.")
    print("\n💡 To configure manually:")
    print("   1. Make sure Java is installed")
    print("   2. Download Stanford POS Tagger from:")
    print("      https://nlp.stanford.edu/software/tagger.shtml#Download")
    print("   3. In Step 3 above, set:")
    print("      stanford_pos_path = '/path/to/stanford-postagger-full-2020-11-17'")
    print("      stanford_language_path = 'models/english-left3words-distsim.tagger'")
    print("   4. Re-run this cell (Step 4)")
    stanford_test_passed = None
    
else:
    print("\n✓ All prerequisites met. Running Stanford tagger test...")
    print(f"\nThis may take several minutes (Stanford tagger is ~100x slower than spaCy)")
    print(f"Processing {len([f for f in os.listdir(CHILDES_DATA_DIR) if f.endswith('.txt')])} files...")
        
    try:
        import time
        start_time = time.time()
        
        results_stanford = prepare_transcripts(
            input_files=CHILDES_DATA_DIR,
            output_file_directory=OUTPUT_DIR_STANFORD,
            run_spell_check=False,
            minwords=2,
            add_additional_tags=True,
            tagger_type='stanford',  # Use Stanford
            stanford_pos_path=stanford_pos_path,
            stanford_language_path=stanford_language_path,
            stanford_batch_size=50,  # Process in batches for better performance
            input_as_directory=True
        )
        
        end_time = time.time()
        processing_time = end_time - start_time
        
        print(f"\n✓ Stanford preprocessing complete!")
        print(f"  Time taken: {processing_time:.1f} seconds ({processing_time/60:.1f} minutes)")
        print(f"  Total utterances processed: {len(results_stanford)}")
        
        # Check if Stanford tags were actually created
        sample_stanford_tags = ast.literal_eval(results_stanford['tagged_stan_token'].iloc[0])
        if sample_stanford_tags:
            print(f"  ✓ Stanford tags successfully generated")
            stanford_test_passed = True
        else:
            print(f"  ✗ Stanford tags are empty")
            stanford_test_passed = False
            
    except Exception as e:
        print(f"\n✗ Stanford preprocessing failed: {e}")
        import traceback
        traceback.print_exc()
        stanford_test_passed = False
        results_stanford = None

### Compare NLTK tags vs Stanford Tags

In [ ]:
# ============================================================
# COMPARISON: NLTK vs Stanford POS Tags
# ============================================================
if results_stanford is not None:
    print("="*60)
    print("COMPREHENSIVE COMPARISON: NLTK vs Stanford POS Tags")
    print("="*60)
    
    total_agreements = 0
    total_tokens = 0
    per_utterance_agreements = []
    
    # Calculate agreement across all utterances
    for idx in range(len(results_stanford)):
        sample_row = results_stanford.iloc[idx]
        
        nltk_tags = ast.literal_eval(sample_row['tagged_token'])
        stanford_tags = ast.literal_eval(sample_row['tagged_stan_token'])
        
        if nltk_tags and stanford_tags and len(nltk_tags) == len(stanford_tags):
            agreements = sum(1 for (_, t1), (_, t2) in zip(nltk_tags, stanford_tags) if t1 == t2)
            total_agreements += agreements
            total_tokens += len(nltk_tags)
            
            # Track per-utterance agreement
            utterance_pct = (agreements / len(nltk_tags)) * 100
            per_utterance_agreements.append(utterance_pct)
    
    # Overall statistics
    overall_agreement_pct = (total_agreements / total_tokens * 100) if total_tokens > 0 else 0
    
    print(f"\n📊 OVERALL STATISTICS:")
    print(f"   Total tokens compared: {total_tokens}")
    print(f"   Agreements: {total_agreements}")
    print(f"   Disagreements: {total_tokens - total_agreements}")
    print(f"   Overall Agreement: {overall_agreement_pct:.1f}%")
    
    if per_utterance_agreements:
        import numpy as np
        print(f"\n   Per-utterance agreement:")
        print(f"      Mean: {np.mean(per_utterance_agreements):.1f}%")
        print(f"      Median: {np.median(per_utterance_agreements):.1f}%")
        print(f"      Min: {np.min(per_utterance_agreements):.1f}%")
        print(f"      Max: {np.max(per_utterance_agreements):.1f}%")
    
    # Show detailed examples
    print("\n" + "="*60)
    print("DETAILED EXAMPLES (First 3 utterances with >5 words)")
    print("="*60)
    
    examples_shown = 0
    for idx in range(len(results_stanford)):
        if examples_shown >= 3:
            break
            
        sample_row = results_stanford.iloc[idx]
        nltk_tags = ast.literal_eval(sample_row['tagged_token'])
        stanford_tags = ast.literal_eval(sample_row['tagged_stan_token'])
        
        if len(nltk_tags) > 5 and len(stanford_tags) > 5:
            examples_shown += 1
            
            print(f"\n--- Example {examples_shown} ---")
            print(f"Source: {sample_row.get('file', 'unknown')}")
            print(f"Participant: {sample_row.get('participant', 'unknown')}")
            print(f"Utterance: {sample_row['content']}\n")
            print(f"{'Word':<15} {'NLTK':<10} {'Stanford':<10} {'Match':<8}")
            print("-" * 48)
            
            agreements = 0
            disagreements = []
            
            for (word_n, tag_n), (word_s, tag_s) in zip(nltk_tags, stanford_tags):
                match = "✓" if tag_n == tag_s else "✗"
                print(f"{word_n:<15} {tag_n:<10} {tag_s:<10} {match:<8}")
                
                if tag_n == tag_s:
                    agreements += 1
                else:
                    disagreements.append((word_n, tag_n, tag_s))
            
            total = len(nltk_tags)
            print(f"\nAgreement: {agreements}/{total} ({100*agreements/total:.1f}%)")
            
            if disagreements:
                print(f"Disagreements: {len(disagreements)}")
                for word, nltk_tag, stanford_tag in disagreements[:3]:
                    print(f"  • '{word}': NLTK={nltk_tag}, Stanford={stanford_tag}")
    
    # Identify most common disagreements
    print("\n" + "="*60)
    print("MOST COMMON TAG DISAGREEMENTS (NLTK vs Stanford)")
    print("="*60)
    
    disagreement_counts = {}
    for idx in range(len(results_stanford)):
        sample_row = results_stanford.iloc[idx]
        nltk_tags = ast.literal_eval(sample_row['tagged_token'])
        stanford_tags = ast.literal_eval(sample_row['tagged_stan_token'])
        
        if nltk_tags and stanford_tags:
            for (word, t1), (_, t2) in zip(nltk_tags, stanford_tags):
                if t1 != t2:
                    key = f"NLTK:{t1} vs Stanford:{t2}"
                    if key not in disagreement_counts:
                        disagreement_counts[key] = []
                    disagreement_counts[key].append(word)
    
    # Show top 10 disagreements
    if disagreement_counts:
        sorted_disagreements = sorted(disagreement_counts.items(), 
                                     key=lambda x: len(x[1]), 
                                     reverse=True)
        
        print("\nTop 10 tag disagreement patterns:")
        for i, (pattern, words) in enumerate(sorted_disagreements[:10], 1):
            example_words = ', '.join(list(set(words))[:3])
            print(f"{i:2}. {pattern:<35} (n={len(words):3}) Examples: {example_words}")
    else:
        print("\n✓ Perfect agreement! No disagreements found.")
    
    print("\n" + "="*60)

### Compare spaCy tags vs Stanford tags

In [ ]:
# ============================================================
# COMPARISON: spaCy vs Stanford POS Tags
# ============================================================
if results_spacy is not None and results_stanford is not None:
    print("="*60)
    print("COMPREHENSIVE COMPARISON: spaCy vs Stanford POS Tags")
    print("="*60)
    
    # First, verify we're comparing the same files
    spacy_files = set(results_spacy['file'].unique())
    stanford_files = set(results_stanford['file'].unique())
    common_files = spacy_files & stanford_files
    
    print(f"\nFiles in spaCy results: {len(spacy_files)}")
    print(f"Files in Stanford results: {len(stanford_files)}")
    print(f"Files in common: {len(common_files)}")
    
    if not common_files:
        print("\n✗ No common files found! Cannot compare.")
    else:
        total_agreements = 0
        total_tokens = 0
        per_utterance_agreements = []
        
        # Process each file that appears in both results
        for file_name in sorted(common_files):
            spacy_data = results_spacy[results_spacy['file'] == file_name].reset_index(drop=True)
            stanford_data = results_stanford[results_stanford['file'] == file_name].reset_index(drop=True)
            
            # Compare row by row
            for idx in range(min(len(spacy_data), len(stanford_data))):
                spacy_tags = ast.literal_eval(spacy_data.iloc[idx]['tagged_spacy_token'])
                stanford_tags = ast.literal_eval(stanford_data.iloc[idx]['tagged_stan_token'])
                
                if spacy_tags and stanford_tags and len(spacy_tags) == len(stanford_tags):
                    agreements = sum(1 for (_, t1), (_, t2) in zip(spacy_tags, stanford_tags) if t1 == t2)
                    total_agreements += agreements
                    total_tokens += len(spacy_tags)
                    
                    # Track per-utterance agreement
                    utterance_pct = (agreements / len(spacy_tags)) * 100
                    per_utterance_agreements.append(utterance_pct)
        
        # Overall statistics
        overall_agreement_pct = (total_agreements / total_tokens * 100) if total_tokens > 0 else 0
        
        print(f"\n📊 OVERALL STATISTICS:")
        print(f"   Total tokens compared: {total_tokens}")
        print(f"   Agreements: {total_agreements}")
        print(f"   Disagreements: {total_tokens - total_agreements}")
        print(f"   Overall Agreement: {overall_agreement_pct:.1f}%")
        
        if per_utterance_agreements:
            import numpy as np
            print(f"\n   Per-utterance agreement:")
            print(f"      Mean: {np.mean(per_utterance_agreements):.1f}%")
            print(f"      Median: {np.median(per_utterance_agreements):.1f}%")
            print(f"      Min: {np.min(per_utterance_agreements):.1f}%")
            print(f"      Max: {np.max(per_utterance_agreements):.1f}%")
        
        # Show detailed examples
        print("\n" + "="*60)
        print("DETAILED EXAMPLES (First 3 utterances with >5 words)")
        print("="*60)
        
        examples_shown = 0
        for file_name in sorted(common_files):
            if examples_shown >= 3:
                break
                
            spacy_data = results_spacy[results_spacy['file'] == file_name].reset_index(drop=True)
            stanford_data = results_stanford[results_stanford['file'] == file_name].reset_index(drop=True)
            
            for idx in range(min(len(spacy_data), len(stanford_data))):
                if examples_shown >= 3:
                    break
                    
                spacy_row = spacy_data.iloc[idx]
                stanford_row = stanford_data.iloc[idx]
                
                spacy_tags = ast.literal_eval(spacy_row['tagged_spacy_token'])
                stanford_tags = ast.literal_eval(stanford_row['tagged_stan_token'])
                
                if len(spacy_tags) > 5 and len(stanford_tags) > 5:
                    examples_shown += 1
                    
                    print(f"\n--- Example {examples_shown} ---")
                    print(f"Source: {spacy_row.get('file', 'unknown')}")
                    print(f"Participant: {spacy_row.get('participant', 'unknown')}")
                    print(f"Utterance: {spacy_row['content']}\n")
                    print(f"{'Word':<15} {'spaCy':<10} {'Stanford':<10} {'Match':<8}")
                    print("-" * 48)
                    
                    agreements = 0
                    disagreements = []
                    
                    for (word_s, tag_s), (word_st, tag_st) in zip(spacy_tags, stanford_tags):
                        match = "✓" if tag_s == tag_st else "✗"
                        print(f"{word_s:<15} {tag_s:<10} {tag_st:<10} {match:<8}")
                        
                        if tag_s == tag_st:
                            agreements += 1
                        else:
                            disagreements.append((word_s, tag_s, tag_st))
                    
                    total = len(spacy_tags)
                    print(f"\nAgreement: {agreements}/{total} ({100*agreements/total:.1f}%)")
                    
                    if disagreements:
                        print(f"Disagreements: {len(disagreements)}")
                        for word, spacy_tag, stanford_tag in disagreements[:3]:
                            print(f"  • '{word}': spaCy={spacy_tag}, Stanford={stanford_tag}")
        
        # Identify most common disagreements
        print("\n" + "="*60)
        print("MOST COMMON TAG DISAGREEMENTS (spaCy vs Stanford)")
        print("="*60)
        
        disagreement_counts = {}
        for file_name in common_files:
            spacy_data = results_spacy[results_spacy['file'] == file_name].reset_index(drop=True)
            stanford_data = results_stanford[results_stanford['file'] == file_name].reset_index(drop=True)
            
            for idx in range(min(len(spacy_data), len(stanford_data))):
                spacy_tags = ast.literal_eval(spacy_data.iloc[idx]['tagged_spacy_token'])
                stanford_tags = ast.literal_eval(stanford_data.iloc[idx]['tagged_stan_token'])
                
                if spacy_tags and stanford_tags:
                    for (word, t1), (_, t2) in zip(spacy_tags, stanford_tags):
                        if t1 != t2:
                            key = f"spaCy:{t1} vs Stanford:{t2}"
                            if key not in disagreement_counts:
                                disagreement_counts[key] = []
                            disagreement_counts[key].append(word)
        
        # Show top 10 disagreements
        if disagreement_counts:
            sorted_disagreements = sorted(disagreement_counts.items(), 
                                         key=lambda x: len(x[1]), 
                                         reverse=True)
            
            print("\nTop 10 tag disagreement patterns:")
            for i, (pattern, words) in enumerate(sorted_disagreements[:10], 1):
                example_words = ', '.join(list(set(words))[:3])
                print(f"{i:2}. {pattern:<35} (n={len(words):3}) Examples: {example_words}")
        else:
            print("\n✓ Perfect agreement! No disagreements found.")
        
        print("\n" + "="*60)

---
## TEST 3: Integration with Alignment Analysis

Test that preprocessed files work correctly with the alignment analysis scripts.

In [ ]:
print("="*60)
print("TEST 3: Integration with Alignment Analysis")
print("="*60)

# Step 1: Import and initialize the alignment analyzer
print("\nStep 1: Initializing alignment analyzer...")
print("-" * 40)

from align_test.alignment import LinguisticAlignment

analyzer = LinguisticAlignment(
    alignment_type="lexsyn",  # Using lexical-syntactic alignment
    cache_dir=os.path.join(OUTPUT_DIR_ALIGNMENT, "cache")
)

print("✓ Analyzer initialized (LexicalSyntacticAlignment)")


import os 
import pandas as pd
import ast






TEST 3: Integration with Alignment Analysis

Step 1: Initializing alignment analyzer...
----------------------------------------


/Users/ndd697/Desktop/Github-Projects/llm-linguistic-alignment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'os' is not defined

In [ ]:
# Initialize alignment analyzer
print("Initializing alignment analyzer...")

analyzer = LinguisticAlignment(
    alignment_type="lexsyn",
    cache_dir=os.path.join(OUTPUT_DIR_ALIGNMENT, "cache")
)

print("✓ Analyzer initialized")

In [ ]:
# Run alignment analysis on preprocessed data
print("\nRunning alignment analysis...")
print(f"Input folder: {OUTPUT_DIR_BASIC}")
print(f"Output folder: {OUTPUT_DIR_ALIGNMENT}")

alignment_results = analyzer.analyze_folder(
    folder_path=OUTPUT_DIR_BASIC,
    output_directory=OUTPUT_DIR_ALIGNMENT,
    lag=1,
    max_ngram=2,
    ignore_duplicates=True,
    add_stanford_tags=False  # Using NLTK-only preprocessed data
)

print(f"\n✓ Alignment analysis complete!")
print(f"Utterance pairs analyzed: {len(alignment_results)}")

In [ ]:
# Examine alignment results
print("Alignment Results:")
print(f"Shape: {alignment_results.shape}")
print(f"\nColumns: {alignment_results.columns.tolist()}")

alignment_results.head()

In [ ]:
# Check for expected alignment metrics
expected_metrics = [
    'lexical_tok1_cosine',
    'lexical_lem1_cosine', 
    'pos_tok1_cosine',
    'pos_lem1_cosine',
    'lexical_master_cosine',
    'syntactic_master_cosine'
]

print("Expected Alignment Metrics:")
print("-" * 40)

found_metrics = []
for metric in expected_metrics:
    present = metric in alignment_results.columns
    status = "✓" if present else "✗"
    print(f"{status} {metric}")
    if present:
        found_metrics.append(metric)

print(f"\nFound {len(found_metrics)}/{len(expected_metrics)} expected metrics")

In [ ]:
# Show sample alignment scores
if found_metrics:
    print("\nSample Alignment Scores:")
    print("="*60)
    
    sample = alignment_results.iloc[0]
    
    print(f"Source: {sample['source_file']}")
    print(f"Participant: {sample['participant']}")
    print(f"Content: {sample['content']}")
    print(f"\nAlignment Scores:")
    
    for metric in found_metrics:
        if metric in sample:
            value = sample[metric]
            print(f"  {metric}: {value:.4f}" if pd.notna(value) else f"  {metric}: NaN")

### TEST 3 Summary

In [ ]:
print("="*60)
print("TEST 3 SUMMARY: Alignment Integration")
print("="*60)

test3_passed = len(alignment_results) > 0 and len(found_metrics) >= 4

if test3_passed:
    print("\n✓ TEST 3 PASSED: Integration with alignment analysis works!")
    print("\nPreprocessed files are fully compatible with alignment analysis.")
    print(f"Successfully analyzed {len(alignment_results)} utterance pairs.")
else:
    print("\n✗ TEST 3 FAILED: Integration issues detected.")
    print("Please review the test results above.")

---
## TEST 4: Check Output Files

Verify that the saved output files on disk are correct.

In [ ]:
print("="*60)
print("TEST 4: Output Files on Disk")
print("="*60)

# Check basic output directory
print(f"\nBasic output directory: {OUTPUT_DIR_BASIC}")
basic_files = [f for f in os.listdir(OUTPUT_DIR_BASIC) if f.endswith('.txt')]
print(f"Files created: {len(basic_files)}")
for f in basic_files:
    size_kb = os.path.getsize(os.path.join(OUTPUT_DIR_BASIC, f)) / 1024
    print(f"  - {f} ({size_kb:.1f} KB)")

In [ ]:
# Load and verify a saved file
if basic_files:
    test_file = os.path.join(OUTPUT_DIR_BASIC, basic_files[0])
    print(f"\nVerifying saved file: {basic_files[0]}")
    
    # Load from disk
    saved_df = pd.read_csv(test_file, sep='\t', encoding='utf-8')
    print(f"✓ Loaded {len(saved_df)} rows from disk")
    
    # Quick format check
    token_str = saved_df['token'].iloc[0]
    print(f"\nToken column type: {type(token_str)}")
    print(f"Token value: {token_str[:80]}...")
    
    # Parse check
    try:
        token_list = ast.literal_eval(token_str)
        print(f"✓ Successfully parsed to: {type(token_list).__name__}")
        print(f"  Contents: {token_list}")
    except Exception as e:
        print(f"✗ Parse failed: {e}")

---
## Final Summary

Overall test results and next steps.

In [ ]:
print("\n" + "="*60)
print("FINAL TEST SUMMARY")
print("="*60)

# Collect results
test_results = {
    "TEST 1: Basic Preprocessing (NLTK)": test1_passed,
    "TEST 2: spaCy Integration": test2_passed if test2_passed is not None else "SKIPPED",
    "TEST 3: Alignment Integration": test3_passed
}

# Print results
for test_name, result in test_results.items():
    if result == "SKIPPED":
        print(f"⊘ {test_name}: SKIPPED")
    elif result:
        print(f"✓ {test_name}: PASSED")
    else:
        print(f"✗ {test_name}: FAILED")

# Overall assessment
passed_tests = [r for r in test_results.values() if r is True]
failed_tests = [r for r in test_results.values() if r is False]

print(f"\nResults: {len(passed_tests)} passed, {len(failed_tests)} failed")

if len(failed_tests) == 0:
    print("\n" + "="*60)
    print("🎉 ALL TESTS PASSED!")
    print("="*60)
    print("\nThe refactored prepare_transcripts.py is working correctly!")
    print("\nYou can now:")
    print("  1. Use prepare_transcripts with your own data")
    print("  2. Run alignment analysis on preprocessed output")
    print("  3. Generate baseline comparisons with surrogates")
else:
    print("\n" + "="*60)
    print("⚠️  SOME TESTS FAILED")
    print("="*60)
    print("\nPlease review the failed tests above.")

---
## Bonus: Quick Preprocessing Example

Once tests pass, here's how to preprocess your own data.

In [ ]:
# Example: Preprocess with spaCy (recommended)
# Uncomment and modify paths for your own data

# from align_test.prepare_transcripts_refactored import prepare_transcripts

# my_results = prepare_transcripts(
#     input_files="/path/to/my/raw/transcripts",
#     output_file_directory="/path/to/my/preprocessed/output",
#     run_spell_check=True,
#     minwords=2,
#     add_stanford_tags=True,
#     stanford_tagger_type='spacy',  # Recommended: fast and accurate
#     save_concatenated_dataframe=True
# )

# print(f"Preprocessed {len(my_results)} utterances!")

---
## Bonus: Full Pipeline Example

Complete workflow from raw data to alignment results.

In [ ]:
# Example: Complete pipeline
# Uncomment to run on your own data

# # Step 1: Preprocess
# preprocessed = prepare_transcripts(
#     input_files="./my_raw_data",
#     output_file_directory="./my_preprocessed",
#     add_stanford_tags=True,
#     stanford_tagger_type='spacy'
# )

# # Step 2: Analyze alignment
# from align_test.alignment import LinguisticAlignment

# analyzer = LinguisticAlignment(alignment_types=["lexsyn", "fasttext"])
# results = analyzer.analyze_folder(
#     folder_path="./my_preprocessed",
#     output_directory="./my_results",
#     lag=1,
#     max_ngram=2,
#     add_stanford_tags=True  # Use spaCy tags from preprocessing
# )

# # Step 3: Generate baseline
# baseline = analyzer.analyze_baseline(
#     input_files="./my_preprocessed",
#     output_directory="./my_results",
#     lag=1,
#     max_ngram=2,
#     add_stanford_tags=True,
#     id_separator="-",
#     condition_label="cond",
#     dyad_label="dyad"
# )

# print("Complete pipeline finished!")